In [1]:
import tensorflow as tf
from tensorflow import keras
import itertools
from utils import *

2024-08-21 07:48:39.652303: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 07:48:39.769435: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 07:48:39.797814: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def custom_loss(fk_weight, fk_loss_type, exp_scale=1.0, huber_delta=0.01):
    def loss_fn(y_true, y_pred):
        angle_loss = tf.keras.losses.Huber(delta=huber_delta)(y_true, y_pred)
        
        # Forward kinematics loss
        fk_true = forward_kinematics_tf(y_true)
        fk_pred = forward_kinematics_tf(y_pred)
        fk_loss = tf.keras.losses.Huber(delta=huber_delta)(fk_true, fk_pred)
        
        # Combine losses
        total_loss = angle_loss + fk_weight * fk_loss
        return total_loss
    return loss_fn

def create_model(config):
    model = keras.Sequential()
    
    # Input layer
    model.add(keras.layers.Input(shape=(3,)))
    
    # Hidden layers
    for units in config['architecture']:
        model.add(keras.layers.Dense(units))
        model.add(keras.layers.Activation(config['activation']))
    
    # Output layer
    model.add(keras.layers.Dense(3, activation='tanh'))
    
    # Create optimizer with gradient clipping
    optimizer = keras.optimizers.Adam(
        learning_rate=config['initial_learning_rate'],
        clipnorm=config['clip_norm']
    )
    model.compile(optimizer=optimizer, loss='mse')  # Using MSE as a generic loss function
    return model


def run_single_experiment(config):
    (train_inputs, train_outputs), (test_inputs, test_outputs), input_mean, input_std = load_and_preprocess_data()
    
    valid_inputs, test_inputs, valid_outputs, test_outputs = train_test_split(
        test_inputs, test_outputs, test_size=0.5, random_state=42
    )
    
    mlflow.set_experiment(config['experiment_name'])
    
    with mlflow.start_run(run_name=config['model_name']):
        mlflow.log_params(config)
        
        steps_per_epoch = len(train_inputs) // config['batch_size']
        total_steps = steps_per_epoch * config['epochs']
        warmup_steps = int(0.1 * total_steps)
        
        model = create_model(config)
        
        model_summary = io.StringIO()
        model.summary(print_fn=lambda x: model_summary.write(x + '\n'))
        mlflow.log_text(model_summary.getvalue(), "model_summary.txt")
        
        lr_scheduler = CosineDecayWithWarmupCallback(
            config['initial_learning_rate'],
            warmup_steps,
            total_steps
        )
        verbose_logging = VerboseLoggingCallback()
        lr_logger = LearningRateLogger()
        
        callbacks = [lr_scheduler, verbose_logging, lr_logger]
        
        callback_names = [callback.__class__.__name__ for callback in callbacks]
        mlflow.log_param("callbacks", ", ".join(callback_names))
        
        start_time = time.time()
        history = model.fit(
            train_inputs, train_outputs,
            epochs=config['epochs'],
            batch_size=config['batch_size'],
            validation_data=(valid_inputs, valid_outputs),
            callbacks=callbacks
        )
        training_time = time.time() - start_time
        
        for epoch, (loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
            mlflow.log_metric("train_loss", loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
        
        mlflow.log_metric("training_time", training_time)
        
        mlflow.tensorflow.log_model(model, "model")
        
        mlflow.log_text(verbose_logging.get_output(), "training_output.txt")
        
        errors, true_xyz, predicted_xyz = evaluate_model(model, test_inputs, test_outputs, input_mean, input_std)
        
        mlflow.log_metric("mean_error", np.mean(errors))
        mlflow.log_metric("median_error", np.median(errors))
        mlflow.log_metric("90th_percentile_error", np.percentile(errors, 90))
        mlflow.log_metric("max_error", np.max(errors))
        
        true_vs_pred_plot_path = f"./Figures/Regularization/{config['model_name']}_true_vs_predicted.png"
        plot_true_vs_predicted(true_xyz, predicted_xyz, f"{config['model_name']} Model: True vs Predicted", save_path=true_vs_pred_plot_path)
        mlflow.log_artifact(true_vs_pred_plot_path)
        
        error_dist_plot_path = f"./Figures/Regularization/{config['model_name']}_error_distribution.png"
        plot_error_distribution(errors, f"{config['model_name']} Model: Error Distribution", save_path=error_dist_plot_path)
        mlflow.log_artifact(error_dist_plot_path)
        
        print(f"\n{config['model_name']} Model:")
        print(f"Mean Error: {np.mean(errors):.4f}")
        print(f"Median Error: {np.median(errors):.4f}")
        print(f"90th Percentile Error: {np.percentile(errors, 90):.4f}")
        print(f"Max Error: {np.max(errors):.4f}")
        print(f"Training Time: {training_time:.2f} seconds")
        
        return {
            'model': model,
            'history': history,
            'errors': errors,
            'true_xyz': true_xyz,
            'predicted_xyz': predicted_xyz,
            'training_time': training_time
        }

In [3]:
# Define architectures
architectures = [
    [64, 128, 64],                     # 1. Simple 3-layer
    [128, 256, 128],                   # 2. Wider 3-layer
    [64, 128, 256, 128, 64],           # 3. 5-layer pyramid
    [256, 256, 256],                   # 4. Deep 3-layer
    [512, 256, 128, 64],               # 5. 4-layer funnel
    [64, 128, 256, 256, 128, 64],      # 6. 6-layer hourglass
    [128, 128, 128, 128],              # 7. 4-layer constant width
    [64, 128, 256, 512, 256, 128, 64], # 8. 7-layer large hourglass
    [32, 64, 128, 256, 512],           # 9. 5-layer expanding
    [1024, 512, 256, 128, 64]          # 10. 5-layer large funnel
]

# Create configurations
configs = []
learning_rates = [0.001, 0.01, 0.04, 0.1]
activations = ['relu', 'swish']
clip_norms = [1.0, 5.0]  # Added gradient clipping norms

for idx, architecture in enumerate(architectures, 1):
    for learning_rate, activation, clip_norm in itertools.product(learning_rates, activations, clip_norms):
        config = {
            "model_name": f"Arch{idx}_LR{learning_rate}_{activation}_Clip{clip_norm}",
            "architecture": architecture,
            "initial_learning_rate": learning_rate,
            "activation": activation,
            "clip_norm": clip_norm,
            "epochs": 100,
            "batch_size": 65536,
            "experiment_name": "Inverse Kinematics Architecture - Gradient Clipping"
        }
        configs.append(config)

# Print the total number of configurations
print(f"Total number of configurations: {len(configs)}")

Total number of configurations: 160


In [4]:
# Print a sample configuration
print("\nSample configuration:")
print(configs[0])


Sample configuration:
{'model_name': 'Arch1_LR0.001_relu_Clip1.0', 'architecture': [64, 128, 64], 'initial_learning_rate': 0.001, 'activation': 'relu', 'clip_norm': 1.0, 'epochs': 100, 'batch_size': 65536, 'experiment_name': 'Inverse Kinematics Architecture - Gradient Clipping'}


In [5]:
# Run experiments
for config in configs:
    run_single_experiment(config)

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 317, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 410, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 1341, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 1334, in _read_helper
    result = read_yaml(root, file_name)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/utils/file_utils.py", line 309, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.MissingConfigException: Yaml file '/tf/workdir/mlruns/mlruns/meta.yaml' does not

Epoch 1/100
 26/138 [====>.........................] - ETA: 0s - loss: 0.3812

2024-08-21 07:48:46.182723: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


138/138 [==============================] - 2s 8ms/step - loss: 0.3285 - val_loss: 0.2424
Epoch 2/100
138/138 [==============================] - 1s 7ms/step - loss: 0.1911 - val_loss: 0.1622
Epoch 3/100
138/138 [==============================] - 1s 7ms/step - loss: 0.1497 - val_loss: 0.1384
Epoch 4/100
138/138 [==============================] - 1s 7ms/step - loss: 0.1305 - val_loss: 0.1240
Epoch 5/100
138/138 [==============================] - 1s 7ms/step - loss: 0.1197 - val_loss: 0.1161
Epoch 6/100
138/138 [==============================] - 1s 7ms/step - loss: 0.1131 - val_loss: 0.1097
Epoch 7/100
138/138 [==============================] - 1s 7ms/step - loss: 0.1061 - val_loss: 0.1024
Epoch 8/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0986 - val_loss: 0.0950
Epoch 9/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0930 - val_loss: 0.0917
Epoch 10/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0909 - val_loss: 0.0902
E

Epoch 40/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0844 - val_loss: 0.0846
Epoch 41/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0844 - val_loss: 0.0845
Epoch 42/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0843 - val_loss: 0.0845
Epoch 43/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0843 - val_loss: 0.0844
Epoch 44/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0842 - val_loss: 0.0843
Epoch 45/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0842 - val_loss: 0.0843
Epoch 46/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0842 - val_loss: 0.0843
Epoch 47/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0841 - val_loss: 0.0842
Epoch 48/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0841 - val_loss: 0.0843
Epoch 49/100
138/138 [==============================] - 1s 7ms/step - loss: 0.0840

...

138/138 [==============================] - 4s 31ms/step - loss: 1.3329 - val_loss: 1.3325
Epoch 78/100
138/138 [==============================] - 4s 31ms/step - loss: 1.3329 - val_loss: 1.3325
Epoch 79/100
138/138 [==============================] - 4s 31ms/step - loss: 1.3329 - val_loss: 1.3325
Epoch 80/100
138/138 [==============================] - 4s 31ms/step - loss: 1.3329 - val_loss: 1.3325
Epoch 81/100
138/138 [==============================] - 4s 31ms/step - loss: 1.3329 - val_loss: 1.3325
Epoch 82/100
138/138 [==============================] - 4s 31ms/step - loss: 1.3329 - val_loss: 1.3325
Epoch 83/100
138/138 [==============================] - 4s 31ms/step - loss: 1.3329 - val_loss: 1.3325
Epoch 84/100
138/138 [==============================] - 4s 32ms/step - loss: 1.3329 - val_loss: 1.3325
Epoch 85/100
138/138 [==============================] - 4s 31ms/step - loss: 1.3329 - val_loss: 1.3325
Epoch 86/100
138/138 [==============================] - 4s 31ms/step - loss: 1.3329 - 

2024/08/21 16:20:43 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmp84i3yljo/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp84i3yljo/model/data/model/assets
2024/08/21 16:20:48 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


8/8 [==============================] - 0s 10ms/step

Arch10_LR0.1_swish_Clip5.0 Model:
Mean Error: 3.4148
Median Error: 3.6067
90th Percentile Error: 4.0952
Max Error: 4.3023
Training Time: 433.08 seconds
